<h1 id="tocheading">Spring 2018 NLP Class Project: Neural Machine Translation</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import spacy
import pdb
import os
from underthesea import word_tokenize
import jieba
import numpy as np
import math
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

# running on cpu
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

/Users/atakanokan/anaconda/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/atakanokan/anaconda/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/atakanokan/anaconda/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/atakanokan/anaconda/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 0.19.0 when using version 0.19.1. This might 

In [3]:
# ! pip install spacy && python -m spacy download en

## Part 0: Project Overview

The goal of this project is to build a neural machine translation system and experience how recent advances have made their way. Each team will build the following sequence of neural translation systems for two language pairs, __Vietnamese (Vi)→English (En)__ and __Chinese (Zh)→En__ (prepared corpora is be provided):

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.
4. [Optional] Build either or both fully self-attention translation system or/and multilingual translation system.

## Part 1: Data Upload & Preprocessing

In [53]:
PAD_index = 0
SOS_index = 1 # start of sentence
UNK_index = 2 # 2 = unk
EOS_index = 3 # end of sentence


class Lang:
    """
    Vocabulary:
        self.word2index = {'Khoa': 2,'hoc': 3,'ang': 4,...}
    Frequency Table:
        self.word2count = {'Khoa': 104,'hoc': 8952,'ang': 13887,...}
    """
    def __init__(self, name):
        self.name = name
        self.word2index = {"<PAD>":PAD_index,
                           "<SOS>":SOS_index,
                           "<UNK>":UNK_index, 
                           "<EOS>":EOS_index}
        self.word2count = {"<PAD>":0,
                           "<SOS>":0,
                           "<UNK>":0, 
                           "<EOS>":0}
        self.index2word = {PAD_index:"<PAD>",
                           SOS_index: "<SOS>",
                           UNK_index:"<UNK>", 
                           EOS_index: "<EOS>"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        # class addWord for every token in the sentence
        # seperated by " "
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        # adds the new word to the vocabulary
        if word not in self.word2index:
            # sets frequency to 1 & increments vocab size 
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1                    
        else:
            # just increments the count if the word is already in the vocab
            self.word2count[word] += 1

In [54]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    """About "NFC" and "NFD": 
    
    For each character, there are two normal forms: normal form C 
    and normal form D. Normal form D (NFD) is also known as canonical 
    decomposition, and translates each character into its decomposed form. 
    Normal form C (NFC) first applies a canonical decomposition, then composes 
    pre-combined characters again.
    
    About unicodedata.category: 
    
    Returns the general category assigned to the Unicode character 
    unichr as string."""
    
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


In [55]:
# Trim
def normalizeString(s):
    # removes blankspaces at the beginning and the end of the string
    s = unicodeToAscii(s.strip())
    # 
    s = re.sub(r"([.!?])", r" \1", s)
    # 
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    
    return s

In [86]:
def readLangs(lang1, lang2, reverse=False,
             dataset="train"):
    
    """Takes as input;
    - lang1, lang2: either (vi, en) or (zh, en)
    - dataset: one of ("train","dev","test")"""
    
#     print("Reading lines...")
    print("\nPreparing the "+ str(dataset) +" Dataset of " + str(lang1) + " to " + str(lang2))
    
    eos = [".","?","!","\n"]
    
    # Read the pretokenized lang1 file and split into lines
    lang1_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang1), encoding="utf-8").\
        read().strip().split("\n")
        
    # Read the lang2 file and split into lines
    lang2_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang2), encoding="utf-8").\
        read().strip().split("\n")
        
    # Examples of Pretokenized Sentences
    random_sent_no = random.randint(0,len(lang1_lines)-1)
    print("\nExample of Language #1 sentence: " + str(lang1_lines[random_sent_no]))
    print("Example of Language #2 sentence: " + str(lang2_lines[random_sent_no]))
    
    # create sentence pairs (lists of length 2 that consist of string pairs)
    # e.g. ["And we &apos;re going to tell you some stories from the sea here in video .",
    #       "我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  "]
    # check if there are the same number of sentences in each set
    assert len(lang1_lines) == len(lang2_lines), "Two languages must have the same number of sentences. "+ str(len(lang1_lines)) + " sentences were passed for " + str(lang1) + "." + str(len(lang2_lines)) + " sentences were passed for " + str(lang2)+"."
    print("\nNumber of sentences in Language #1 = " + str(len(lang1_lines)))
    print("Number of sentences in Language #2 = " + str(len(lang2_lines)))
    
    # normalize if not Chinese, Chinese normalization is already handeled
    # add <EOS> tag at the end of the sentence for chinese
    if lang1 == "zh":
        lang1_lines = [s + "<EOS>" for s in lang1_lines]
    else:
        # replace .?!\n with <EOS> tag for Vietnamese and English
        lang1_lines = [normalizeString(s).replace(".","<EOS>").\
                       replace("?","<EOS>").replace("!","<EOS>").replace("\n","<EOS>") for s in lang1_lines]
    lang2_lines = [normalizeString(s).replace(".","<EOS>").\
                       replace("?","<EOS>").replace("!","<EOS>").replace("\n","<EOS>") for s in lang2_lines]
    
    # construct pairs
    pair_ran = range(len(lang1_lines))
    pairs = [[lang1_lines[i]] + [lang2_lines[i]] for i in pair_ran]
    
#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [87]:
def prepareData(lang1, lang2, reverse=False, dataset="train"):
    
    input_lang, output_lang, pairs = readLangs(lang1 = lang1, 
                                               lang2 = lang2, 
                                               reverse = reverse, 
                                               dataset = dataset)
    
#     print("Read %s sentence pairs" % len(pairs))
#     print("Trimmed to %s sentence pairs" % len(pairs))

#     print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        
    print("\nVocabulary sizes:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    # Checking number of <EOS> tags in the sentences
    print("\nnumber of sentence pairs = " + str(len(pairs)))
    print("number of <EOS> tags in the source sentences = " + str(input_lang.word2count["<EOS>"]))
    print("number of <EOS> tags in the target sentences = " + str(output_lang.word2count["<EOS>"]))

    print("\nRandom preprocessed sentence pair:")
    print(random.choice(pairs))
    
    return input_lang, output_lang, pairs

In [90]:
input_lang, output_lang, pairs = prepareData(lang1 = 'vi', 
                                             lang2 = 'en', 
                                             reverse = False, 
                                             dataset = "train")



Preparing the train Dataset of vi to en

Example of Language #1 sentence: Tôi đã ngồi bên cạnh một trong luật_sư rất can_đảm của chúng_tôi , và tôi nói , " Làm thế_nào để chúng_ta có_thể biến điều_luật này thành hiện_thực ? Làm thế_nào chúng_ta có_thể đảm_bảo rằng nó được thi_hành ? "
Example of Language #2 sentence: And I was sitting side by side with one of our very courageous lawyers , and said , &quot; How can we get this out ? How can we make sure that this is implemented ?

Number of sentences in Language #1 = 133317
Number of sentences in Language #2 = 133317

Vocabulary sizes:
vi 16143
en 47567

number of sentence pairs = 133317
number of <EOS> tags in the source sentences = 138953
number of <EOS> tags in the target sentences = 148908

Random preprocessed sentence pair:
['Va toi phan ung Khong <EOS> Tai sao su bi an hay quy mo lon lao cua vu tru lai can phai i cung voi mot trang thai cam xuc huyen bi <EOS>', 'And I go quot No <EOS> quot Why does that sense of mystery that sens

In [72]:
input_lang.word2index

{'<PAD>': 0,
 '<SOS>': 1,
 '<UNK>': 2,
 '<EOS>': 3,
 'Khoa': 4,
 'hoc': 5,
 'ang': 6,
 'sau': 7,
 'mot': 8,
 'tieu': 9,
 'e': 10,
 've': 11,
 'khi': 12,
 'hau': 13,
 'Trong': 14,
 'phut': 15,
 'chuyen': 16,
 'gia': 17,
 'hoa': 18,
 'quyen': 19,
 'Rachel': 20,
 'Pike': 21,
 'gioi': 22,
 'thieu': 23,
 'so': 24,
 'luoc': 25,
 'nhung': 26,
 'no': 27,
 'luc': 28,
 'khoa': 29,
 'miet': 30,
 'mai': 31,
 'tao': 32,
 'bao': 33,
 'bien': 34,
 'oi': 35,
 'cung': 36,
 'voi': 37,
 'oan': 38,
 'nghien': 39,
 'cuu': 40,
 'cua': 41,
 'minh': 42,
 'hang': 43,
 'ngan': 44,
 'nguoi': 45,
 'a': 46,
 'cong': 47,
 'hien': 48,
 'cho': 49,
 'du': 50,
 'an': 51,
 'nay': 52,
 'bay': 53,
 'mao': 54,
 'hiem': 55,
 'qua': 56,
 'rung': 57,
 'tim': 58,
 'kiem': 59,
 'thong': 60,
 'tin': 61,
 'phan': 62,
 'tu': 63,
 'then': 64,
 'chot': 65,
 'Toi': 66,
 'muon': 67,
 'cac': 68,
 'ban': 69,
 'biet': 70,
 'su': 71,
 'to': 72,
 'lon': 73,
 'gop': 74,
 'lam': 75,
 'nen': 76,
 'dong': 77,
 'tit': 78,
 'thuong': 79,
 'thay'

In [73]:
input_lang.word2count

{'<PAD>': 0,
 '<SOS>': 0,
 '<UNK>': 0,
 '<EOS>': 138953,
 'Khoa': 104,
 'hoc': 8952,
 'ang': 13887,
 'sau': 5911,
 'mot': 49113,
 'tieu': 1943,
 'e': 22485,
 've': 18759,
 'khi': 14456,
 'hau': 1552,
 'Trong': 1857,
 'phut': 884,
 'chuyen': 8085,
 'gia': 8074,
 'hoa': 5193,
 'quyen': 2033,
 'Rachel': 18,
 'Pike': 3,
 'gioi': 6504,
 'thieu': 1295,
 'so': 10381,
 'luoc': 231,
 'nhung': 44550,
 'no': 17845,
 'luc': 4834,
 'khoa': 2212,
 'miet': 18,
 'mai': 1096,
 'tao': 6492,
 'bao': 8197,
 'bien': 3299,
 'oi': 12582,
 'cung': 12828,
 'voi': 22470,
 'oan': 2548,
 'nghien': 2217,
 'cuu': 3023,
 'cua': 38031,
 'minh': 8123,
 'hang': 4198,
 'ngan': 1884,
 'nguoi': 26945,
 'a': 25028,
 'cong': 10462,
 'hien': 6025,
 'cho': 21271,
 'du': 6441,
 'an': 6940,
 'nay': 26190,
 'bay': 3443,
 'mao': 213,
 'hiem': 1010,
 'qua': 10007,
 'rung': 558,
 'tim': 4684,
 'kiem': 2547,
 'thong': 6853,
 'tin': 4917,
 'phan': 8197,
 'tu': 19549,
 'then': 62,
 'chot': 277,
 'Toi': 12741,
 'muon': 6838,
 'cac': 22

In [74]:
output_lang.word2index

{'<PAD>': 0,
 '<SOS>': 1,
 '<UNK>': 2,
 '<EOS>': 3,
 'Rachel': 4,
 'Pike': 5,
 'The': 6,
 'science': 7,
 'behind': 8,
 'a': 9,
 'climate': 10,
 'headline': 11,
 'In': 12,
 'minutes': 13,
 'atmospheric': 14,
 'chemist': 15,
 'provides': 16,
 'glimpse': 17,
 'of': 18,
 'the': 19,
 'massive': 20,
 'scientific': 21,
 'effort': 22,
 'bold': 23,
 'headlines': 24,
 'on': 25,
 'change': 26,
 'with': 27,
 'her': 28,
 'team': 29,
 'one': 30,
 'thousands': 31,
 'who': 32,
 'contributed': 33,
 'taking': 34,
 'risky': 35,
 'flight': 36,
 'over': 37,
 'rainforest': 38,
 'in': 39,
 'pursuit': 40,
 'data': 41,
 'key': 42,
 'molecule': 43,
 'I': 44,
 'apos': 45,
 'd': 46,
 'like': 47,
 'to': 48,
 'talk': 49,
 'you': 50,
 'today': 51,
 'about': 52,
 'scale': 53,
 'that': 54,
 'goes': 55,
 'into': 56,
 'making': 57,
 'see': 58,
 'paper': 59,
 'Headlines': 60,
 'look': 61,
 'this': 62,
 'when': 63,
 'they': 64,
 'have': 65,
 'do': 66,
 'and': 67,
 'air': 68,
 'quality': 69,
 'or': 70,
 'smog': 71,
 'They'

In [75]:
output_lang.word2count

{'<PAD>': 0,
 '<SOS>': 0,
 '<UNK>': 0,
 '<EOS>': 148908,
 'Rachel': 20,
 'Pike': 4,
 'The': 6667,
 'science': 763,
 'behind': 417,
 'a': 54259,
 'climate': 314,
 'headline': 26,
 'In': 2758,
 'minutes': 549,
 'atmospheric': 12,
 'chemist': 20,
 'provides': 51,
 'glimpse': 36,
 'of': 60341,
 'the': 103138,
 'massive': 169,
 'scientific': 237,
 'effort': 137,
 'bold': 65,
 'headlines': 43,
 'on': 13313,
 'change': 1612,
 'with': 13280,
 'her': 2257,
 'team': 420,
 'one': 8316,
 'thousands': 464,
 'who': 5289,
 'contributed': 27,
 'taking': 550,
 'risky': 26,
 'flight': 107,
 'over': 2632,
 'rainforest': 36,
 'in': 38689,
 'pursuit': 43,
 'data': 1194,
 'key': 313,
 'molecule': 141,
 'I': 43694,
 'apos': 64433,
 'd': 1767,
 'like': 8303,
 'to': 65797,
 'talk': 1759,
 'you': 33041,
 'today': 1583,
 'about': 11111,
 'scale': 370,
 'that': 46612,
 'goes': 704,
 'into': 4150,
 'making': 911,
 'see': 5220,
 'paper': 400,
 'Headlines': 2,
 'look': 2780,
 'this': 21400,
 'when': 5876,
 'they': 1

In [89]:
input_lang, output_lang, pairs = prepareData(lang1 = 'zh', 
                                             lang2 = 'en', 
                                             reverse = False, 
                                             dataset = "train")


Preparing the train Dataset of zh to en

Example of Language #1 sentence: 我 现在 明白   我 从来 从来不 不是 一个   也 不是 另 一个  
Example of Language #2 sentence: I see now , I never was one and not the other .

Number of sentences in Language #1 = 213376
Number of sentences in Language #2 = 213376

Vocabulary sizes:
zh 89203
en 59328

number of sentence pairs = 213376
number of <EOS> tags in the source sentences = 212842
number of <EOS> tags in the target sentences = 240005

Random preprocessed sentence pair:
['接下 接下来 下来 发生 的 是    她 说    请坐下 请坐下来 坐下 下来    安静 点   听 我 说 的 做   遵守 守规矩 规矩    抓紧 抓紧时间 时间   注意 一下    要 乖 点 像 个 女孩   <EOS>', 'And what happens is she says quot Please sit down be quiet do what you apos re told follow the rules manage your time focus be a girl <EOS> quot ']
['当 这些 被 测量    反而   我们 发现 这个 速度 基本 基本上 是 一个 常数    作为 距离 的 函数  <EOS>', 'When those measurements are made instead what we find is that the speed is basically constant as a function of distance <EOS>']


### 1.1 Vietnamese to English

In [91]:
# Format: languagepair_language_dataset
# Train 
vien_vi_train, vien_en_train, vi_en_train_pairs = prepareData('vi', 'en', False, dataset="train")
# Dev 
vien_vi_dev, vien_en_dev, vi_en_dev_pairs = prepareData('vi', 'en', False, dataset="dev")
# Test
vien_vi_test, vien_en_test, vi_en_test_pairs = prepareData('vi', 'en', False, dataset="test")


Preparing the train Dataset of vi to en

Example of Language #1 sentence: Điều này chính là 33 % còn thiếu của việc thành công bình_đẳng trong công_việc cho phụ_nữ chứ không phải là_vì chúng_ta thiếu năng_lực hay khả_năng cũng không phải vì chúng_ta thiếu lời_khuyên .
Example of Language #2 sentence: This is that missing 33 percent of the career success equation for women , not because it &apos;s missing in our capabilities or abilities , but because it &apos;s missing in the advice that we &apos;re given .

Number of sentences in Language #1 = 133317
Number of sentences in Language #2 = 133317

Vocabulary sizes:
vi 16143
en 47567

number of sentence pairs = 133317
number of <EOS> tags in the source sentences = 138953
number of <EOS> tags in the target sentences = 148908

Random preprocessed sentence pair:
['Mot trong nhung nha tien phong vi du la tien sy Anthony Atala va ong ay ang lam cong viec phan tich te bao e tao nen cac bo phan co the ruot than <EOS> <EOS> <EOS>', 'So one of th

### 1.2 Chinese to English

In [92]:
# Format: languagepair_language_dataset
# Train 
zhen_zh_train, zhen_en_train, zh_en_train_pairs = prepareData('zh', 'en', False, dataset="train")
# Dev 
zhen_zh_dev, zhen_en_dev, zh_en_dev_pairs = prepareData('zh', 'en', False, dataset="dev")
# Test
zhen_zh_test, zhen_en_test, zh_en_test_pairs = prepareData('zh', 'en', False, dataset="test")


Preparing the train Dataset of zh to en

Example of Language #1 sentence: 直到 午夜    接着 再次 入睡    大约 从 凌晨 2 点 直到 日出  
Example of Language #2 sentence: until midnight and then again , they sleep from about 2 : 00 a.m. until sunrise .

Number of sentences in Language #1 = 213376
Number of sentences in Language #2 = 213376

Vocabulary sizes:
zh 89203
en 59328

number of sentence pairs = 213376
number of <EOS> tags in the source sentences = 212842
number of <EOS> tags in the target sentences = 240005

Random preprocessed sentence pair:
['然后 把 这些 喷 桩 从 沙丘 里 拔出 拔出来 出来    以 沙 为 模子    我们 几乎 能 在 沙丘 内部 创造 任何 我们 想要 的 形状  <EOS>', 'We then pull the piles up through the dune and we apos re able to create almost any conceivable shape inside of the sand with the sand acting as a mold as we go up <EOS>']

Preparing the dev Dataset of zh to en

Example of Language #1 sentence:   我 最后 爬出 了 矿井 得以 回家   而 这些 被 奴役 的 矿工 可能 永远 没有 回家 的 那 一天  
Example of Language #2 sentence: I got to climb out of that hole , and 

#### Chinese word2index & index2word

In [93]:
zhen_zh_train.word2index["格"]

10482

In [95]:
zhen_zh_train.index2word[10482]

'格'

#### Vietnamese word2index & index2word

In [96]:
vien_vi_train.word2index["Hamburger"]

6751

In [97]:
vien_vi_train.index2word[6751]

'Hamburger'

#### English (Chinese) word2index & index2word

In [98]:
zhen_en_train.word2index["translate"]

1450

In [100]:
zhen_en_train.index2word[1450]

'translate'

#### English (Vietnamese) word2index & index2word

In [101]:
vien_en_train.word2index["machine"]

847

In [102]:
vien_en_train.index2word[847]

'machine'

### 1.4 Prepare Dataloaders

In [110]:
"we" in zhen_en_train.word2index

True

In [111]:
zhen_en_train.word2index["we"]

51

In [112]:
PAD_IDX = PAD_index # 0 
SOS_IDX = SOS_index # 1
UNK_IDX = UNK_index # 2
EOS_IDX = EOS_index # 3

# convert token to id in the dataset
def token2index_dataset(paired_tokens, 
                        lang1_token2id_vocab,
                        lang2_token2id_vocab):
    """Takes as input:
    - paired_tokens: a list of sentence pairs that consist of 
                    source & target lang sentences.
    - lang1_token2id_vocab: token2index vocabulary for the first language. 
                            Get by method Lang_dataset.word2index
    - lang2_token2id_vocab: token2index vocabulary for the second language. 
                            Get by method Lang_dataset.word2index
                            
    Returns:
    - indices_data_lang_1, indices_data_lang2: A list of lists where each sub-list holds corresponding indices for each
                                               token in the sentence."""
    indices_data_lang_1, indices_data_lang_2 = [], []
    vocabs = [lang1_token2id_vocab, lang2_token2id_vocab]
    
    # lang1
    for t in range(len(paired_tokens)):
        # replaces token with UNK_IDX if the token is not in vocab
        index_list = [vocabs[0][token] if token in vocabs[0]\
                                    else UNK_IDX for token in paired_tokens[t][0]] 
        indices_data_lang_1.append(index_list)
    # lang2
    for t in range(len(paired_tokens)):
        index_list =  [vocabs[1][token] if token in vocabs[1] \
                                    else UNK_IDX for token in paired_tokens[t][1]] 
        indices_data_lang_2.append(index_list)
     
    # Documenting the transformation
    random_sentence = random.randint(0, len(paired_tokens)-1)
    print("Random sentence pair = " + str(random_sentence))
    print(paired_tokens[random_sentence])
    print("Random token2indexized sentence pair:")
    print("Indexed source sentence:" + str(indices_data_lang_1[random_sentence]))
    print("Indexed target sentence:" + str(indices_data_lang_2[random_sentence]))
       
    return indices_data_lang_1, indices_data_lang_2

In [115]:
# train indices
print("\n\nTraining Datasets")
print("Zh -> En")
zhen_zh_train_indices, zhen_en_train_indices = token2index_dataset(zh_en_train_pairs,
                                                                   zhen_zh_train.word2index,
                                                                   zhen_en_train.word2index)

print("\nVi -> En")
vien_vi_train_indices, vien_en_train_indices = token2index_dataset(vi_en_train_pairs,
                                                                   vien_vi_train.word2index,
                                                                   vien_en_train.word2index)

# dev indices
print("\n\nDev Datasets")
print("Zh -> En")
zhen_zh_dev_indices, zhen_en_dev_indices = token2index_dataset(zh_en_dev_pairs,
                                                               zhen_zh_dev.word2index,
                                                               zhen_en_dev.word2index)
print("\nVi -> En")
vien_vi_dev_indices, vien_en_dev_indices = token2index_dataset(vi_en_dev_pairs,
                                                               vien_vi_dev.word2index,
                                                               vien_en_dev.word2index)

# test indices
print("\n\nTest Datasets")
print("Zh -> En")
zhen_zh_test_indices, zhen_en_test_indices = token2index_dataset(zh_en_test_pairs,
                                                                 zhen_zh_test.word2index,
                                                                 zhen_en_test.word2index)
print("\nVi -> En")
vien_vi_test_indices, vien_en_test_indices = token2index_dataset(vi_en_test_pairs,
                                                                 vien_vi_test.word2index,
                                                                 vien_en_test.word2index)



Training Datasets
Zh -> En
Random sentence pair = 106426
['三天 后   我 从 我 的 朋友 兼 伙伴 兼 同事 比尔 尔克 克林顿 哪里 收到 一封   手写   精美 兼 很 长 的 一封 一封信 封信   内容 是    艾 尔   祝 新 餐馆 开张   <EOS>', 'Three days later I got a nice long handwritten letter from my friend and partner and colleague Bill Clinton saying quot Congratulations on the new restaurant Al <EOS> quot ']
Random token2indexized sentence pair:
Indexed source sentence:[4911, 4265, 2, 619, 2, 2, 2, 50, 2, 309, 2, 50, 2, 6, 2, 46940, 41876, 2, 13091, 2, 70631, 31598, 2, 13091, 2, 11658, 1997, 2, 161, 8108, 2, 8108, 6732, 2, 6732, 7771, 11418, 2, 237, 116, 2, 9557, 348, 2, 679, 8620, 2, 2, 2, 706, 6348, 2, 2, 2, 44009, 13454, 2, 13091, 2, 329, 2, 246, 2, 6, 2, 679, 8620, 2, 679, 8620, 9653, 2, 8620, 9653, 2, 2, 2, 2477, 12897, 2, 32, 2, 2, 2, 2, 6333, 2, 8108, 2, 2, 2, 8521, 2, 215, 2, 24277, 10850, 2, 5210, 397, 2, 2, 2, 2, 5869, 16125, 5790, 2]
Indexed target sentence:[5690, 2, 46403, 8296, 8296, 2, 1736, 159, 8786, 25, 2, 14281, 159, 269, 8296, 46

In [120]:
def index2token(language,index_list):
    word_sentence = []
    for index in index_list:
        word_sentence.append(language.index2word[index])
        
    return word_sentence

In [123]:
try_index = [2277, 2, 219, 2, 2, 2, 2, 3564, 2, 219, 2, 2, 3221, 2, 86, 2, 3564, 2, 2, 2, 3221, 2, 219, 2, 3221, 2, 3221, 2, 1503, 219, 2, 2, 2, 2, 2, 2, 86, 2, 2, 2, 2, 2, 2, 2, 2, 86, 2, 2, 3221, 2, 3221, 2, 1503, 2, 2, 738, 1503, 2, 2, 25, 2, 2, 86, 3221, 3221, 2, 219, 2, 2, 86, 2, 2, 2, 3121, 2788, 2]

print(index2token(language = vien_en_train,
            index_list = try_index))

['gamer', '<UNK>', 'This', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'Singer', '<UNK>', 'This', '<UNK>', '<UNK>', 'certified', '<UNK>', 'put', '<UNK>', 'Singer', '<UNK>', '<UNK>', '<UNK>', 'certified', '<UNK>', 'This', '<UNK>', 'certified', '<UNK>', 'certified', '<UNK>', 'send', 'This', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'put', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'put', '<UNK>', '<UNK>', 'certified', '<UNK>', 'certified', '<UNK>', 'send', '<UNK>', '<UNK>', 'contextualized', 'send', '<UNK>', '<UNK>', 'on', '<UNK>', '<UNK>', 'put', 'certified', 'certified', '<UNK>', 'This', '<UNK>', '<UNK>', 'put', '<UNK>', '<UNK>', '<UNK>', 'Hawaiian', 'accepting', '<UNK>']


In [30]:
# check length
# train
print ("Chinese training sentence count = "+str(len(zhen_zh_train_indices)))
print ("Chinese-English (En) training sentence count = "+str(len(zhen_en_train_indices)))
print ("\nVietnamese training sentence count = "+str(len(vien_vi_train_indices)))
print ("Vietnamese-English (En) training sentence count = "+str(len(vien_en_train_indices)))
# dev
print ("\nChinese dev sentence count = "+str(len(zhen_zh_dev_indices)))
print ("Chinese-English (En) dev sentence count = "+str(len(zhen_en_dev_indices)))
print ("\nVietnamese dev sentence count = "+str(len(vien_vi_dev_indices)))
print ("Vietnamese-English (En) dev sentence count = "+str(len(vien_en_dev_indices)))
# test
print ("\nChinese test sentence count = "+str(len(zhen_zh_test_indices)))
print ("Chinese-English (En) test sentence count = "+str(len(zhen_en_test_indices)))
print ("\nVietnamese test sentence count = "+str(len(vien_vi_test_indices)))
print ("Vietnamese-English (En) test sentence count = "+str(len(vien_en_test_indices)))

Chinese training sentence count = 213376
Chinese-English (En) training sentence count = 213376

Vietnamese training sentence count = 133317
Vietnamese-English (En) training sentence count = 133317

Chinese dev sentence count = 1261
Chinese-English (En) dev sentence count = 1261

Vietnamese dev sentence count = 1268
Vietnamese-English (En) dev sentence count = 1268

Chinese test sentence count = 1397
Chinese-English (En) test sentence count = 1397

Vietnamese test sentence count = 1553
Vietnamese-English (En) test sentence count = 1553


#### Dataloader

In [35]:
from torch.utils.data import Dataset

In [42]:
## TODO 

MAX_SENTENCE_LENGTH = 15
BATCH_SIZE = 32

# zhen token2index vocabs
zhen_zh_train_token2id = zhen_zh_train.word2index
zhen_en_train_token2id = zhen_en_train.word2index

# vien token2index vocabs
vien_vi_train_token2id = vien_vi_train.word2index
vien_en_train_token2id = vien_en_train.word2index

class TranslationDataset():
    """
    Class that represents a train/dev/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, 
                 data_source, # training indices data of the source language
                 data_target, # training indices data of the target language
                 token2id_source=None, # token2id dict of the source language
                 token2id_target=None  # token2id dict of the target language
                ):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.source_sentences = data_source
        self.target_sentences = data_target
        print("self.source_sentences = " + str(self.source_sentences[0]))
        print("self.target_sentences = " + str(self.target_sentences[0]))
        
        self.token2id_source = token2id_source
        self.token2id_target = token2id_target
        # prints the mandarin token -> # dictionary
        """
        It also contains wrong tokenized words & Latin Alphabet words too
        '11<EOS>': 13275, '为啥': 13276, '指责': 13277, '超载': 13278, 
        '复杂度': 13279, '小玩意': 13280, '小玩意儿': 13281, '此行': 13282, 
        '断面': 13283, '强制': 13284, '制发': 13285, '花瓶': 13286,
        '糖': 13287, '年缴': 13288, '缴纳': 13289, '会费': 13290,
        '99': 13291, 'Photoshop': 13292, '4000<EOS>': 13293, 
        '升级': 13294, '佯': 13295, '谬': 13296, 'Microsoft': 13297, 
        'Word': 13298, '文字': 13299,
        """
#         print("self.token2id_source = " + str(self.token2id_source))
        # prints the english token -> # dictionary
#         print("self.token2id_target = " + str(self.token2id_target))        

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, batch_index):

#         source_word_idx, target_word_idx = [], []
        source_mask, target_mask = [], []
        
        for index in self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                source_mask.append(0)
            else:
                source_mask.append(1)
                
        for index in self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                target_mask.append(0)
            else:
                target_mask.append(1)
        
        source_indices = self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        target_indices = self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        
        source_list = [source_indices, source_mask, len(source_indices)]
        target_list = [target_indices, target_mask, len(target_indices)]
        
        return source_list + target_list

    
def translation_collate(batch, max_sentence_length):
    """
    Customized function for DataLoader that dynamically pads the 
    batch so that all data have the same length
    """
    source_data, target_data = [], []
    source_mask, target_mask = [], []
    source_lengths, target_lengths = [], []

    for datum in batch:
        source_lengths.append(datum[2])
        target_lengths.append(datum[5])
        
        # PAD
        source_data_padded = np.pad(np.array(datum[0]), 
                                    pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", 
                                    constant_values=0)
        source_data.append(source_data_padded)
        
        source_mask_padded = np.pad(np.array(datum[1]), 
                                    pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", 
                                    constant_values=0)
        source_mask.append(source_mask_padded)
        
        target_data_padded = np.pad(np.array(datum[3]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                                mode="constant", constant_values=0)
        target_data.append(target_data_padded)
        
        target_mask_padded = np.pad(np.array(datum[4]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                               mode="constant", constant_values=0)
        target_mask.append(target_mask_padded)
        
    ind_dec_order = np.argsort(source_lengths)[::-1]
    source_data = np.array(source_data)[ind_dec_order]
    target_data = np.array(target_data)[ind_dec_order]
    source_mask = np.array(source_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    target_mask = np.array(target_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    source_lengths = np.array(source_lengths)[ind_dec_order]
    target_lengths = np.array(target_lengths)[ind_dec_order]
    
    source_list = [torch.from_numpy(source_data), 
               torch.from_numpy(source_mask).float(), source_lengths]
    target_list = [torch.from_numpy(target_data), 
               torch.from_numpy(target_mask).float(), target_lengths]
        
    return source_list + target_list

print("Loading zhen_train_dataset:")
zhen_train_dataset = TranslationDataset(zhen_zh_train_indices,
                                       zhen_en_train_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_train_loader = torch.utils.data.DataLoader(dataset=zhen_train_dataset,
                               batch_size=BATCH_SIZE,
                               collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                               shuffle=False)

print("\nLoading zhen_dev_dataset:")
zhen_dev_dataset = TranslationDataset(zhen_zh_dev_indices,
                                       zhen_en_dev_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_dev_loader = torch.utils.data.DataLoader(dataset=zhen_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

print("\nLoading vien_train_dataset:")
vien_train_dataset = TranslationDataset(vien_vi_train_indices,
                                       vien_en_train_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_train_loader = torch.utils.data.DataLoader(dataset=vien_train_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

print("\nLoading vien_dev_dataset:")
vien_dev_dataset = TranslationDataset(vien_vi_dev_indices,
                                       vien_en_dev_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_dev_loader = torch.utils.data.DataLoader(dataset=vien_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)


Loading zhen_train_dataset:
self.source_sentences = [321, 7912, 2, 7912, 310, 2, 4, 2, 1586, 23701, 2, 2, 2, 275, 49581, 2, 2, 2, 5915, 6331, 2, 2, 5868, 16124, 5789, 2]
self.target_sentences = [4216, 16909, 49096, 8295, 2, 16909, 2107, 2, 268, 2, 8295, 2, 1735, 8295, 8295, 10558, 2, 1263, 28417, 8295, 158, 2107, 23]

Loading zhen_dev_dataset:
self.source_sentences = [2, 2, 1232, 1232, 2, 4, 2, 55, 91, 2, 2, 2, 2, 2695, 650, 2, 650, 14, 2, 766, 678, 2, 4782, 2, 2, 2, 281, 2, 2, 2, 27, 2802, 2, 430, 729, 2, 14, 2, 2, 2, 2, 16, 2, 4733, 5952, 2, 2, 2, 2, 2, 2, 2, 2]
self.target_sentences = [2, 2, 2, 2, 2, 3, 2, 2, 30, 199, 2, 3, 2, 2, 2, 419, 2, 419, 2, 2, 2, 2, 2, 30, 2, 1211, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 419, 2, 2, 2, 1211, 2, 2, 2, 51, 2, 2, 51, 2, 2, 2, 199, 2, 2, 2, 1109, 2, 2, 2, 2, 2, 2, 2, 2, 1211, 2, 2, 419, 2, 2, 2, 2, 2, 199, 2, 2, 2, 2, 2, 2, 2]

Loading vien_train_dataset:
self.source_sentences = [3018, 2843, 137, 44, 2, 2843, 137, 1836, 2, 44, 1830, 5219, 2, 1082, 44, 888,

In [43]:
##########################################
#### SAMPLE DATASET - CLEAR OUT LATER ####
##########################################

# zhen_train_dataset = TranslationDataset(zhen_zh_train_indices[:480], # 15 batches
#                                        zhen_en_train_indices[:480],
#                                        token2id_source=zhen_zh_train_token2id,
#                                        token2id_target=zhen_en_train_token2id)

# zhen_train_loader = torch.utils.data.DataLoader(dataset=zhen_train_dataset,
#                                batch_size=BATCH_SIZE,
#                                collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
#                                shuffle=False)

## Part 2: Evaluation Metric

We use BLEU as the evaluation metric. Specifically, we focus on the corpus-level BLEU function. 

The code for BLEU is taken from https://github.com/mjpost/sacreBLEU/blob/master/sacrebleu.py#L1022-L1080

In [44]:
! pip3 install sacrebleu

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [126]:
# import sacrebleu

In [127]:
# TODO

## Part 3: Beam Search Algorithm

In this section, we implement the Beam Search algorithm in Pytorch.

## Part 4: Model

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.

#### Loss Function & Evaluation

In [46]:
# reconstruction loss = binary cross entropy between two (vocab_size x 1) vectors
# used during training, since we can compare the real Y and and the generated Y
# still at each time step of the decoder, we compare up to and including
# the real t-th token and the generated t-th, then optimize

def loss_function(y_hat, y):
    
    """Takes as input;
    - y: correct "log-softmax"(binary vector) that represents the correct t-th token in the target sentence,
                 (vocab_size x 1) vector
    - y_hat: predicted LogSoftmax for the predicted t-th token in the target sentence.
             (vocab_size x 1) vector
    Returns;
    - NLL Loss in training time"""
#     y_hat = torch.log(y_hat) # log softmax
    loss = nn.functional.binary_cross_entropy(y_hat,y)
    
    return loss
    

# generation/inference time - validation loss = BLEU

def compute_BLEU(corpus_hat,corpus):
    ## TODO
    return None


#### Beam Search

### Beam Search

In [48]:
torch.FloatTensor([3,4,2,7,5,3,2]).topk(3)

(tensor([7., 5., 4.]), tensor([3, 4, 1]))

In [49]:
BATCH_SIZE = 32

class BeamSearch(nn.Module):
    
    """network that conducts beam search over the outputs of
     any translator network. The translator networks that can 
     be passed are:
     
     - Translate (for RNN-enc-dec),
     - AttnTranslate (for RNN-enc-dec with attention),
     - CNNtranslate (for CNN-encoder based translation).
     
     The translation networks take care of the encoder-decoder
     choices specific to each task. Please see in below sections."""

    def __init__(self, translator_network, beam_size):
        super().__init__()
        # translator network that returns the logsoftmax
        # over vocabulary size:(vocab_size, 1)
        self.translator_network = translator_network
        self.beam_size = beam_size
        
    def init_search_tree(self, batch_size):
        beam_size = self.beam_size
        self.search_tree = torch.empty(batch_size, beam_size, 1)
        return self
    
    def init_score_tree(self, batch_size):
        beam_size = self.beam_size
        search_tree = self.search_tree
        self.score_tree = torch.zeros(search_tree.size())
        return self
    
    def forward(source_sentence, source_mask, source_lengths,
                target_sentence, target_mask, target_lengths):
        
        self.init_search_tree(BATCH_SIZE)
        self.init_score_tree(BATCH_SIZE)
        
        # at each time step the decoder will give us the logsoftmax
        # of one token (batch_size, vocab_size). 
        output = model(source_sentence, target_sentence,source_mask, 
                       target_mask, source_lengths,target_lengths)
        
        # for each sentence in the batch we get the top k predictions
        # for each token and append it to the search and score trees. 
        for i in range(BATCH_SIZE):
            beam = output[i].topk(beam_size) # (token scores, token indices)
            # cat instead
            self.search_tree[i] = self.search_tree.cat(beam[1]) # cat the indices to the search tree
            self.score_tree[i,:] = beam[0] # append the scores to the score tree 
        
        # we will sum the logs 
        
        

### 2.1: RNN-based Encoder-Decoder without Attention

In [50]:
batch_size = 32

In [54]:
def convert_to_softmax(tensor_of_indices,
                       batch_size,
                       vocab_size = len(zhen_en_train_token2id)):
    """
    - takes as input a time_step vector of the batch (t-th token of each sentence in the batch)
      size: (batch_size, 1)
    - converts it to softmax of (batch_size, vocab_size)
    """
    index_tensor_ = tensor_of_indices.view(-1,1).long()
        
    one_hot = torch.FloatTensor(batch_size, vocab_size).zero_()
    one_hot.scatter_(1, index_tensor_.detach().cpu(), 1)
    
    return one_hot

### Fully self-attention Translation System / Transformer

In [39]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

    
    
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)    

    
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)
    
    
    
    
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
    
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))
    
    
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    
    
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)
    
    
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)
    
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
    
    
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
    
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
    

In [47]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
    
    
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
#             nn.init.xavier_uniform_(p)
            nn.init.xavier_uniform_(p)
    return model






In [48]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [49]:
tmp_model = make_model(10, 10, 2)
tmp_model

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
      

In [50]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [51]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [52]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [53]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [124]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        
        # creates a copy of the Variable
        true_dist = x.data.clone()
        print("true_dist = " + str(true_dist))
        
        # Fills self tensor with the specified value
        true_dist.fill_(self.smoothing / (self.size - 2.0))
        print("true_dist = " + str(true_dist))
        
        # scatter_(dim, index, src) → Tensor
        print("self.confidence = " + str(self.confidence))
        true_dist.scatter_(dim = 1, 
                           index = target.data.unsqueeze(1), 
                           src = self.confidence)
        
        true_dist[:, self.padding_idx] = 0.
        mask = torch.nonzero(target.data == self.padding_idx)
#         if mask.dim() > 0:
        if mask.sum() > 0 and len(mask) > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [125]:
crit = LabelSmoothing(5, 0, 0.1)
def loss(x):
    d = x + 3 * 1
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d],
                                 ])
    #print(predict)
    return crit(Variable(predict.log()),
                 Variable(torch.LongTensor([1]))).data[0]

/Users/atakanokan/anaconda/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [119]:
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)

In [120]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        x_loss = x.contiguous().view(-1, x.size(-1)).float()
        y_loss = y.contiguous().view(-1)
        loss = self.criterion(x_loss, y_loss) / norm
                              
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data[0] * norm

In [121]:
V = 11

criterion = LabelSmoothing(size=V, 
                           padding_idx=0, 
                           smoothing=0.0)

model = make_model(V, V, N=2)

model_opt = NoamOpt(model.src_embed[0].d_model, 
                    1, 
                    400,
                    torch.optim.Adam(model.parameters(), 
                                     lr=0, 
                                     betas=(0.9, 0.98), 
                                     eps=1e-9))

for epoch in range(10):
    model.train()
    run_epoch(data_iter = data_gen(V, 30, 20), 
              model = model, 
              loss_compute = SimpleLossCompute(generator = model.generator, 
                                               criterion = criterion, 
                                               opt = model_opt))
    model.eval()
    print(run_epoch(data_iter = data_gen(V, 30, 5), 
                    model = model, 
                    loss_compute = SimpleLossCompute(generator = model.generator, 
                                                     criterion = criterion, 
                                                     opt = None)))

/Users/atakanokan/anaconda/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


true_dist = tensor([[-1.9340, -4.1510, -2.6695,  ..., -2.6970, -3.7252, -1.2385],
        [-2.3561, -1.1062, -3.0696,  ..., -2.7260, -3.8816, -1.6418],
        [-3.7725, -2.3680, -4.6145,  ..., -4.4205, -4.0762, -1.8362],
        ...,
        [-2.7200, -1.7991, -4.6643,  ..., -1.3054, -5.5390, -1.0155],
        [-3.3538, -1.7163, -2.9694,  ..., -2.2880, -3.2859, -2.1620],
        [-4.2577, -1.2764, -2.6886,  ..., -2.4744, -2.3833, -1.3999]])
true_dist = tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
self.confidence = 1.0


TypeError: scatter_() received an invalid combination of arguments - got (src=float, index=Tensor, dim=int, ), but expected one of:
 * (int dim, Tensor index, Tensor src)
      didn't match because some of the arguments have invalid types: ([32;1mdim=int[0m, [32;1mindex=Tensor[0m, [31;1msrc=float[0m, )
 * (int dim, Tensor index, Number value)
      didn't match because some of the keywords were incorrect: src
